In [6]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

## Deutsch's Algorithm

It solves the Deutsch Problem.

INPUT: $f: \Sigma \rightarrow \Sigma$
OUTPUT: 0 if f is a constant function else 1 if f is balanced

There are 4 possible functions on single bit:
<li>Constant 0</li>
<li>Constant 1</li>
<li>Same as input</li>
<li>bit flip</li>


In [7]:
def deutsch_function(case: int):
    """
    Deutsch's function generates a valid deutsch's function as a QuantumCircuit 
    :param case: integer value for one of the 4 cases discussed
    :return: QuantumCircuit object
    """
    if case not in [1, 2, 3, 4]:
        raise ValueError("`case` must be 1, 2, 3, or 4.")

    f = QuantumCircuit(2)
    if case in [2, 3]:
        f.cx(0, 1)
    if case in [3, 4]:
        f.x(1)
    return f

In [8]:
deutsch_function(3).draw()

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

In [9]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in Deutsch's algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)

    qc.x(n)                                 # Pauli X gate
    qc.h(range(n + 1))                      # Hadamard gate

    qc.barrier()
    qc.compose(function, inplace=True)      # Use the deutsch function
    qc.barrier()

    qc.h(range(n))                          # Hadamard gate
    qc.measure(range(n), range(n))          # Finally measure

    return qc

In [10]:
compile_circuit(
    deutsch_function(3)
).draw()

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

In [11]:
def deutsch_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch function is constant or balanced.
    """
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if measurements[0] == "0":
        return "constant"
    return "balanced"

In [16]:
f = deutsch_function(4)
display(f.draw())
deutsch_algorithm(f)

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

'constant'

In [17]:
f = deutsch_function(3)
display(f.draw())
deutsch_algorithm(f)

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

'balanced'

In [18]:
f = deutsch_function(2)
display(f.draw())
deutsch_algorithm(f)

q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

'balanced'

In [19]:
f = deutsch_function(1)
display(f.draw())
deutsch_algorithm(f)

q_0: 
     
q_1:

'constant'